In [39]:
import pandas as pd
import gensim
from gensim.corpora import Dictionary
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from textblob import TextBlob
import re
from gensim import corpora
from gensim import models

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\annegru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\annegru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
uk_speed = pd.read_excel(r'C:UK_neg_speed.xlsx')
uk_reliability = pd.read_excel(r'UK_neg_reliability.xlsx')

uk_speed

In [7]:
uk_speed = uk_speed['Comment'].tolist()
uk_reliability= uk_reliability['Comment'].tolist()


In [ ]:
uk_reliability

In [27]:

stoplist = set(stopwords.words('english'))
punctuation = set(string.punctuation)
ps = PorterStemmer()

def preprocessing(list):
    preprocessed_comments = []
    for comment in list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        
        tokens = word_tokenize(comment)
        #tokens = [token for token in tokens if token not in stoplist and token not in punctuation]
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        #tokens = [token for token in tokens if token.isnumeric()]
        tokens = [ps.stem(token) for token in tokens]
        
        for token in tokens:
            if token.isnumeric() == True:
                tokens.remove(token)
        
        preprocessed_comment = ' '.join(tokens)
        preprocessed_comments.append(preprocessed_comment)
    
    return preprocessed_comments

In [43]:
stoplist = set(stopwords.words('english'))
punctuation = string.punctuation + '‘’“”' 
ps = PorterStemmer()

def preprocessed_comments(comment_list):
    preprocessed_comments = []
    for comment in comment_list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        tokens = word_tokenize(comment)
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        tokens = [ps.stem(token) for token in tokens]
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        preprocessed_comments.append(tokens)
    return preprocessed_comments

In [ ]:
print(preprocessed_comments(uk_speed))

In [ ]:
print(preprocessed_comments(uk_reliability))

In [ ]:
# finish with the preprocessing function, work on removing numeric values
# move on with next steps, making bowvector to be able to feed it into lda model 

In [46]:
# create a dictionary, each token is given an integer ID
uk_speed_preprocessed = preprocessed_comments(uk_speed)
speed_dictionary = corpora.Dictionary(preprocessed_comments(uk_speed))

uk_reliability_preprocessed = preprocessed_comments(uk_reliability)
reliability_dictionary = corpora.Dictionary(preprocessed_comments(uk_reliability))

print(reliability_dictionary)

Dictionary(2546 unique tokens: ['crucial', 'downtim', 'expens', 'hour', 'internet']...)


In [47]:
# convert tokens into BoW representation

speed_corpus = [speed_dictionary.doc2bow(comment) for comment in uk_speed_preprocessed]
reliability_corpus = [reliability_dictionary.doc2bow(comment) for comment in uk_reliability_preprocessed]

In [48]:
#train lda model
# set identified topics to a desired nr
# show 10 most important words (represented by probability of word belonging to topic) 
# assign label to topic based on most represented words

num_topics = 5
passes = 10
speed_lda = models.LdaModel(speed_corpus, id2word=speed_dictionary, num_topics=num_topics, passes=passes)
reliability_lda = models.LdaModel(reliability_corpus, id2word=reliability_dictionary, num_topics=num_topics, passes=passes)

In [49]:
speed_lda.print_topics()
reliability_lda.print_topics()

[(0,
  '0.039*"internet" + 0.031*"work" + 0.022*"get" + 0.020*"time" + 0.018*"drop" + 0.014*"keep" + 0.014*"day" + 0.013*"broadband" + 0.010*"issu" + 0.010*"home"'),
 (1,
  '0.045*"internet" + 0.028*"servic" + 0.028*"broadband" + 0.026*"drop" + 0.021*"connect" + 0.014*"work" + 0.013*"poor" + 0.012*"custom" + 0.012*"router" + 0.011*"expens"'),
 (2,
  '0.030*"internet" + 0.022*"servic" + 0.016*"work" + 0.015*"time" + 0.012*"custom" + 0.012*"virgin" + 0.011*"intermitt" + 0.010*"call" + 0.010*"issu" + 0.009*"connect"'),
 (3,
  '0.022*"wifi" + 0.020*"broadband" + 0.018*"work" + 0.015*"virgin" + 0.013*"time" + 0.012*"custom" + 0.011*"servic" + 0.011*"price" + 0.010*"issu" + 0.009*"reliabl"'),
 (4,
  '0.029*"issu" + 0.021*"wifi" + 0.019*"broadband" + 0.019*"servic" + 0.016*"router" + 0.015*"get" + 0.015*"time" + 0.010*"phone" + 0.009*"ive" + 0.009*"told"')]

In [50]:
for topic in speed_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

Topic 0:
['speed', 'internet', 'servic', 'virgin', 'custom', 'slow', 'poor', 'get', 'time', 'router']

Topic 1:
['broadband', 'speed', 'servic', 'custom', 'slow', 'tv', 'time', 'get', 'poor', 'drop']

Topic 2:
['speed', 'servic', 'custom', 'price', 'get', 'time', 'good', 'broadband', 'internet', 'contract']

Topic 3:
['speed', 'get', 'pay', 'servic', 'connect', 'internet', 'broadband', 'dont', 'slow', 'virgin']

Topic 4:
['speed', 'internet', 'slow', 'wifi', 'get', 'broadband', 'connect', 'time', 'drop', 'work']



In [ ]:
#topic 0: internet speed & customer service, linking to virgin
#topic 1: broadband speed drops & customer service
#topic 2: broadband & internet, focus on price
#topic 3: slow internet, issues with internet & broadband connectivity, pay
#topic 4: slow wifi & broadband, poor connection, drops at times and doesnt always work

In [51]:
for topic in reliability_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

Topic 0:
['internet', 'work', 'get', 'time', 'drop', 'keep', 'day', 'broadband', 'issu', 'home']

Topic 1:
['internet', 'servic', 'broadband', 'drop', 'connect', 'work', 'poor', 'custom', 'router', 'expens']

Topic 2:
['internet', 'servic', 'work', 'time', 'custom', 'virgin', 'intermitt', 'call', 'issu', 'connect']

Topic 3:
['wifi', 'broadband', 'work', 'virgin', 'time', 'custom', 'servic', 'price', 'issu', 'reliabl']

Topic 4:
['issu', 'wifi', 'broadband', 'servic', 'router', 'get', 'time', 'phone', 'ive', 'told']



In [ ]:
#topic 0: at home internet drops
#topic 1: router, customer service, expensive
#topic 2: related to the company, calling customer service (?), connectivity issues
#topic 3: issues with pricing and wifi/broadband
#topic 4: wifi/broadband, customer service calling on the phone(?)

# CH

In [64]:
def no_stemming_preprocessed_comments(comment_list):
    preprocessed_comments = []
    for comment in comment_list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        tokens = word_tokenize(comment)
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        #tokens = [ps.stem(token) for token in tokens]
        #tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        preprocessed_comments.append(tokens)
    return preprocessed_comments

In [97]:
ch_outages = pd.read_excel(r'C:CH(internet-outages).xlsx')
ch_reliability = pd.read_excel(r'C:CH(internet-reliability).xlsx')

In [98]:
ch_outages = ch_outages['comment'].tolist()
ch_reliability = ch_reliability['comment'].tolist()

In [99]:
ch_outages_preprocessed = no_stemming_preprocessed_comments(ch_outages)
ch_reliability_preprocessed = no_stemming_preprocessed_comments(ch_reliability)

In [100]:
print(ch_reliability_preprocessed)

[['often', 'restart', 'router'], ['wired', 'network', 'reliability'], ['weak', 'connection', 'depending', 'location', 'slow', 'fastest', 'subscription'], ['connection', 'sometimes', 'bad', 'none'], ['improve', 'stability', 'faster', 'internet'], ['unstable', 'micro', 'cut', 'causing', 'big', 'problems', 'skype', 'zoom', 'calls', 'contractual', 'speed', 'real', 'irregular'], ['lately', 'become', 'unstable', 'without', 'connection', 'hours'], ['unstable', 'wifi', 'connection', 'often', 'lost'], ['stability', 'speed', 'poor'], ['since', 'providers', 'taken', 'problem', 'mix', 'internet', 'information'], ['router', 'restarted', 'every', 'days', 'hangs', 'somehow', 'therefore', 'unreliable'], ['game', 'play', 'often', 'play', 'people', 'regularly', 'fall', 'server', 'according', 'forums', 'game', 'due', 'stable', 'line', 'preferably', 'fixed', 'ip', 'address', 'way', 'ive', 'written', 'way', 'computer', 'connected', 'via', 'lan', 'everything', 'work', 'smoothly'], ['network', 'stability', '

In [101]:
ch_outages_dictionary = corpora.Dictionary(ch_outages_preprocessed)
ch_reliability_dictionary = corpora.Dictionary(ch_reliability_preprocessed)

In [102]:
ch_outages_corpus = [ch_outages_dictionary.doc2bow(comment) for comment in ch_outages_preprocessed]
ch_reliability_corpus = [ch_reliability_dictionary.doc2bow(comment) for comment in ch_reliability_preprocessed]

In [103]:
num_topics = 4
passes = 10
ch_outages_lda = models.LdaModel(ch_outages_corpus, id2word=speed_dictionary, num_topics=num_topics, passes=passes)
ch_reliability_lda = models.LdaModel(ch_reliability_corpus, id2word=reliability_dictionary, num_topics=num_topics, passes=passes)

In [104]:
for topic in ch_outages_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

Topic 0:
['well', 'renew', 'older', 'virgin', 'significantli', 'problem', 'devic', 'downstair', 'advisor', 'constraint']

Topic 1:
['regular', 'higher', 'channel', 'claim', 'deal…', 'staff', 'late', 'say', 'aw', 'charg']

Topic 2:
['price', 'use', 'side', 'capabl', 'renew', 'vari', 'back', 'cabl', '60mb', 'suspect']

Topic 3:
['price', 'regular', 'renew', 'channel', 'report', 'vari', 'wors', 'stand', 'run', 'manag']



In [105]:
for topic in ch_reliability_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

Topic 0:
['expens', 'pay', 'never', 'zoom', '£60', 'abysm', 'ago', 'anymor', 'hub', 'drop']

Topic 1:
['make', 'consist', 'connect', 'devic', 'drop', 'mani', 'work', 'show', 'hour', 'period']

Topic 2:
['make', 'consist', 'drop', 'got', 'age', 'broadband', 'also', 'wors', 'put', 'work']

Topic 3:
['work', 'make', 'crucial', 'second', 'server', 'breakdown', 'pass', 'wors', 'internet', 'hub']

